In [ ]:
from decodes.core import *
from decodes.io.jupyter_out import JupyterOut
import math

out = JupyterOut.unit_square( )

# The Raster Family
todo

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P22.jpg" style="width: 200px; display: inline;">


<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P30.jpg" style="width: 200px; display: inline;">


## Rasters

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.C05.jpg" style="width: 200px; display: inline;">

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P07.jpg" style="width: 200px; display: inline;">


<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P31.jpg" style="width: 200px; display: inline;">


### Raster Descendants

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P24.jpg" style="width: 200px; display: inline;">


## Grids

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P25.jpg" style="width: 200px; display: inline;">


### Bounds Objects in Decod.es

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/3.00.D88 Bounds Large.jpg" style="width: 800px; display: inline;">


<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P32.jpg" style="width: 200px; display: inline;">


### Grid Objects in Decod.es

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.C06.jpg" style="width: 200px; display: inline;">

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P33.jpg" style="width: 200px; display: inline;">


## Multiple Inheritance

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P26.jpg" style="width: 200px; display: inline;">


<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P10.jpg" style="width: 200px; display: inline;">


<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P11.jpg" style="width: 200px; display: inline;">
